# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [6]:
rf = RandomForestClassifier()
param = {"n_estimators":[10,150,300],
        "max_depth":[30,60,90,None]}
gs = GridSearchCV(rf,param,cv=5)
gs_fit = gs.fit(X_tfidf_feat,data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,6.699611,0.073819,0.147246,0.004852,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.976661,0.979354,0.975741,0.970350,0.973046,0.975030,0.003089,1
11,14.887513,0.791205,0.219965,0.009049,None,300,"{'max_depth': None, 'n_estimators': 300}",0.978456,0.977558,0.973944,0.969452,0.972147,0.974312,0.003351,2
8,13.392019,0.216893,0.207500,0.005563,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.979354,0.973944,0.968553,0.971249,0.974132,0.003962,3
4,5.727775,0.194877,0.133856,0.002676,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.977558,0.973968,0.973046,0.965858,0.974843,0.973055,0.003901,4
10,9.490975,3.365914,0.229313,0.089810,None,150,"{'max_depth': None, 'n_estimators': 150}",0.976661,0.975763,0.974843,0.967655,0.967655,0.972515,0.004010,5


In [4]:
rf = RandomForestClassifier()
param = {"n_estimators":[10,150,300],
        "max_depth":[30,60,90,None]}
gs = GridSearchCV(rf,param,cv=5)
gs_fit = gs.fit(X_count ,data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,4.576917,0.339848,0.068317,0.003929,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975763,0.972172,0.973046,0.967655,0.965858,0.970899,0.003628,1
7,3.604577,0.014033,0.056917,0.000254,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977558,0.973070,0.972147,0.965858,0.965858,0.970898,0.004504,2
11,14.095836,1.995694,0.238414,0.047763,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.970377,0.973944,0.964960,0.967655,0.970719,0.004204,3
6,0.245554,0.003944,0.006005,0.000071,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.974865,0.968582,0.967655,0.970350,0.967655,0.969821,0.002707,4
8,7.219415,0.027655,0.111031,0.000191,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.967684,0.970350,0.965858,0.964960,0.969282,0.004530,5
